# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f436c299400>
├── 'a' --> tensor([[-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892]])
└── 'x' --> <FastTreeValue 0x7f436c2997f0>
    └── 'c' --> tensor([[-0.5780,  0.3479, -0.8074,  2.2673],
                        [ 0.2615, -0.8109, -0.5532, -0.3995],
                        [ 0.4903,  0.7902, -0.7654, -0.8773]])

In [4]:
t.a

tensor([[-0.3192,  0.5320, -1.2184],
        [ 0.8464,  0.5500, -0.6892]])

In [5]:
%timeit t.a

72.5 ns ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f436c299400>
├── 'a' --> tensor([[ 0.6366, -0.6571,  0.7629],
│                   [ 0.8711,  0.5802, -1.5024]])
└── 'x' --> <FastTreeValue 0x7f436c2997f0>
    └── 'c' --> tensor([[-0.5780,  0.3479, -0.8074,  2.2673],
                        [ 0.2615, -0.8109, -0.5532, -0.3995],
                        [ 0.4903,  0.7902, -0.7654, -0.8773]])

In [7]:
%timeit t.a = new_value

81.8 ns ± 1.57 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892]]),
    x: Batch(
           c: tensor([[-0.5780,  0.3479, -0.8074,  2.2673],
                      [ 0.2615, -0.8109, -0.5532, -0.3995],
                      [ 0.4903,  0.7902, -0.7654, -0.8773]]),
       ),
)

In [10]:
b.a

tensor([[-0.3192,  0.5320, -1.2184],
        [ 0.8464,  0.5500, -0.6892]])

In [11]:
%timeit b.a

65.3 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0711,  2.1855,  1.5258],
               [ 1.0777,  0.4105, -0.8797]]),
    x: Batch(
           c: tensor([[-0.5780,  0.3479, -0.8074,  2.2673],
                      [ 0.2615, -0.8109, -0.5532, -0.3995],
                      [ 0.4903,  0.7902, -0.7654, -0.8773]]),
       ),
)

In [13]:
%timeit b.a = new_value

640 ns ± 6.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 8.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 95.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

289 µs ± 7.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

297 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f42bff50eb0>
├── 'a' --> tensor([[[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]],
│           
│                   [[-0.3192,  0.5320, -1.2184],
│                    [ 0.8464,  0.5500, -0.6892]]])
└── 'x' --> <FastTreeValue 0x7f436c299b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.7 µs ± 460 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f42bffba400>
├── 'a' --> tensor([[-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892],
│                   [-0.3192,  0.5320, -1.2184],
│                   [ 0.8464,  0.5500, -0.6892]])
└── 'x' --> <FastTreeValue 0x7f42c6b88850>
    └── 'c' --> tensor([[-0.5780,  0.3479, -0.8074,  2.2673],
                        [ 0.2615, -0.8109, -0.5532, -0.3995],
                 

In [23]:
%timeit t_cat(trees)

44.1 µs ± 618 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.5 µs ± 876 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]],
       
               [[-0.3192,  0.5320, -1.2184],
                [ 0.8464,  0.5500, -0.6892]]]),
    x: Batch(
           c: tensor([[[-0.5780,  0.3479, -0.8074,  2.2673],
                       [ 0.2615, -0.8109, -0.5532, -0.3995],
                       [ 0.4903,  0.7902, -0.7654, -0.8773]],
         

In [26]:
%timeit Batch.stack(batches)

109 µs ± 779 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892],
               [-0.3192,  0.5320, -1.2184],
               [ 0.8464,  0.5500, -0.6892]]),
    x: Batch(
           c: tensor([[-0.5780,  0.3479, -0.8074,  2.2673],
                      [ 0.2615, -0.8109, -0.5532, -0.3995],
                      [ 0.4903,  0.7902, -0.7654, -0.8773],
                      [-0.5780,  0.3479, -0.8074,  2.2673],
                      [ 0.2615, -

In [28]:
%timeit Batch.cat(batches)

193 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

616 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
